# Data Discovery Demonstration

In this demo we are going to analyze a small dataset that has been extracted from data.gov.
The dataset consists of 6 CSV files and it it 10MB size in total.
The files are related to X and Y and Z

### Pre-Processing: Loading data and creating signatures

In [4]:
import api as API
API.load_precomputed_model()

Loading signature collections...
Done deserialization of signature collections!
Loading signature collections...DONE!
Loading graph...
Done deserialization of signature collection!
Loading graph...DONE!
Loading dataset columns...
Done deserialization of dataset columns!
Loading dataset columns...DONE!


In [5]:
concept = ("STC_2014_STC005_with_ann.csv", "Geography")

In [6]:
discovered_concepts = API.give_related_concepts(concept)

In [8]:
discovered_concepts

[('STC_2014_STC005_with_ann.csv', 'Geography'),
 ('Income_Tax_Components_by_Size_of_Income_by_Place_of_Residence__Beginning_Tax_Year_1999.csv',
  'State'),
 ('The_Tax_Burden_on_Tobacco_Volume_49__1970-2014.csv', 'LocationDesc'),
 ('STC_2014_STC006.US01_with_ann.csv', 'Government'),
 ('SAMHSA_Synar_Reports__Youth_Tobacco_Sales.csv', 'LocationDesc'),
 ('NYS_Liquor_Authority_New_Applications_Received.csv', 'City'),
 ('Income_Tax_Components_by_Size_of_Income_by_Place_of_Residence__Beginning_Tax_Year_1999.csv',
  'Place of Residence'),
 ('STC_2014_STC005_with_ann.csv', 'Geography')]

In [9]:
import pprint

In [10]:
pp = pprint.PrettyPrinter(indent=4)

In [12]:
pp.pprint(discovered_concepts)

[   ('STC_2014_STC005_with_ann.csv', 'Geography'),
    (   'Income_Tax_Components_by_Size_of_Income_by_Place_of_Residence__Beginning_Tax_Year_1999.csv',
        'State'),
    ('The_Tax_Burden_on_Tobacco_Volume_49__1970-2014.csv', 'LocationDesc'),
    ('STC_2014_STC006.US01_with_ann.csv', 'Government'),
    ('SAMHSA_Synar_Reports__Youth_Tobacco_Sales.csv', 'LocationDesc'),
    ('NYS_Liquor_Authority_New_Applications_Received.csv', 'City'),
    (   'Income_Tax_Components_by_Size_of_Income_by_Place_of_Residence__Beginning_Tax_Year_1999.csv',
        'Place of Residence'),
    ('STC_2014_STC005_with_ann.csv', 'Geography')]
